<a href="https://colab.research.google.com/github/FahmidaNourin/CSE445_Machine_Learning/blob/main/Project_Initial.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#connecting with drive

In [55]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


#read in csv file

In [56]:
import numpy as np
import pandas as pd


df = pd.read_csv("/content/drive/MyDrive/Sleep_Research/HMC_Sleep_Stages.csv")

In [57]:
df.head()

,Subject,Sleep_Stage,Epoch,MeanP_Alpha_F4,MedianF_Alpha_F4,MeanF_Alpha_F4,SpectralEdge_Alpha_F4,PeakF_Alpha_F4,MeanP_Beta_F4,MedianF_Beta_F4,...,MeanP_Delta_O2,MedianF_Delta_O2,MeanF_Delta_O2,SpectralEdge_Delta_O2,PeakF_Delta_O2,MeanP_Gamma_O2,MedianF_Gamma_O2,MeanF_Gamma_O2,SpectralEdge_Gamma_O2,PeakF_Gamma_O2
0,SN002,N1,6,0.00034,10.11512,17.85756,17.48293,10.24000,0.00047,17.73268,...,0.00112,0.74927,6.36878,2.87220,0.62439,0.000076,32.71805,43.83220,49.95122,49.95122
1,SN002,N1,7,0.00035,10.73951,17.60780,17.10829,9.11610,0.00048,16.85854,...,0.00070,0.99902,6.86829,3.62146,0.74927,0.000072,32.09366,44.33171,49.95122,49.95122
2,SN002,N1,8,0.00035,9.36585,17.60780,16.98341,9.24098,0.00042,17.35805,...,0.00077,1.49854,6.49366,4.12098,0.74927,0.000069,33.09268,43.95707,49.95122,49.95122
3,SN002,N1,116,0.00054,9.74049,15.98439,15.48488,8.36683,0.00051,14.61073,...,0.00082,0.87415,7.49268,3.49659,0.87415,0.000039,28.34732,43.08293,49.95122,49.95122
4,SN002,N1,117,0.00034,10.11512,16.48390,15.98439,12.23805,0.00034,15.11024,...,0.00084,0.87415,6.86829,2.87220,0.62439,0.000032,28.97171,43.20780,49.95122,49.95122


In [58]:
df = df.dropna().reset_index(drop=True)

In [59]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 108451 entries, 0 to 108450
Data columns (total 78 columns):
 #   Column                 Non-Null Count   Dtype  
---  ------                 --------------   -----  
 0   Subject                108451 non-null  object 
 1   Sleep_Stage            108451 non-null  object 
 2   Epoch                  108451 non-null  int64  
 3   MeanP_Alpha_F4         108451 non-null  float64
 4   MedianF_Alpha_F4       108451 non-null  float64
 5   MeanF_Alpha_F4         108451 non-null  float64
 6   SpectralEdge_Alpha_F4  108451 non-null  float64
 7   PeakF_Alpha_F4         108451 non-null  float64
 8   MeanP_Beta_F4          108451 non-null  float64
 9   MedianF_Beta_F4        108451 non-null  float64
 10  MeanF_Beta_F4          108451 non-null  float64
 11  SpectralEdge_Beta_F4   108451 non-null  float64
 12  PeakF_Beta_F4          108451 non-null  float64
 13  MeanP_Theta_F4         108451 non-null  float64
 14  MedianF_Theta_F4       108451 non-nu

#Converting N1,N2 -> Light_Sleep, N3 -> Deep_Sleep And removing W stage.

In [60]:
df.shape

(108451, 78)

In [61]:
df =df.replace("N1", "Light_Sleep")
df =df.replace("N2", "Light_Sleep")
df =df.replace("N3", "Deep_Sleep")
df =df.replace("R", "REM")
df.drop(df.index[df['Sleep_Stage'] == "W"], inplace = True)


In [62]:
df.shape

(89096, 78)

#Removing columns

In [63]:
df = df.loc[:,df.columns != "Subject"]
df = df.loc[:,df.columns != "Epoch"]

#Feature Selection: ANOVA

In [84]:



from sklearn.datasets import make_classification
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import f_classif
import pandas as pd

X = df.loc[:,df.columns != "Sleep_Stage"]
y = df["Sleep_Stage"]

fs = SelectKBest(score_func=f_classif, k=5)
fit = fs.fit(X,y)
dfscores = pd.DataFrame(fit.scores_)
dfcolumns = pd.DataFrame(X.columns)

featureScores = pd.concat([dfcolumns,dfscores],axis=1)

featureScores.columns = ['Best_columns','Score_ANOVA']

lyst = featureScores.nlargest(10,'Score_ANOVA')

In [85]:
lyst

,Best_columns,Score_ANOVA
2,MeanF_Alpha_F4,1838.921875
3,SpectralEdge_Alpha_F4,1617.329209
51,MedianF_Alpha_O2,862.440302
42,MeanF_Delta_C4,812.708423
27,MeanF_Alpha_C4,794.641118
17,MeanF_Delta_F4,721.002533
1,MedianF_Alpha_F4,694.479074
28,SpectralEdge_Alpha_C4,674.647154
67,MeanF_Delta_O2,655.401827
53,SpectralEdge_Alpha_O2,424.680863


#Encoding target columns (Sleep Stage)

In [64]:
target="Sleep_Stage"

In [65]:
df[target].value_counts()

Light_Sleep    51341
Deep_Sleep     21290
REM            16465
Name: Sleep_Stage, dtype: int64

In [66]:
from sklearn.preprocessing import LabelEncoder
encd = LabelEncoder()

In [67]:
df[target] = encd.fit_transform(df[target])

In [68]:
df[target].value_counts()

1    51341
0    21290
2    16465
Name: Sleep_Stage, dtype: int64

#Spliting Dataset into X(features) and y(target)

In [88]:
X = df.loc[:,df.columns != target]
y = df[target]



#Scaling Dataset

In [89]:
X

,Sleep_Stage,MeanP_Alpha_F4,MedianF_Alpha_F4,MeanF_Alpha_F4,SpectralEdge_Alpha_F4,PeakF_Alpha_F4,MeanP_Beta_F4,MedianF_Beta_F4,MeanF_Beta_F4,SpectralEdge_Beta_F4,...,MeanP_Delta_O2,MedianF_Delta_O2,MeanF_Delta_O2,SpectralEdge_Delta_O2,PeakF_Delta_O2,MeanP_Gamma_O2,MedianF_Gamma_O2,MeanF_Gamma_O2,SpectralEdge_Gamma_O2,PeakF_Gamma_O2
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
89091,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
89092,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
89093,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
89094,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [72]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()

scaler.fit(X)

X[X.columns] =   scaler.transform(X)

<ipython-input-72-fd1652dd9516>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X[X.columns] =   scaler.transform(X)


In [73]:
X

,MeanP_Alpha_F4,MedianF_Alpha_F4,MeanF_Alpha_F4,SpectralEdge_Alpha_F4,PeakF_Alpha_F4,MeanP_Beta_F4,MedianF_Beta_F4,MeanF_Beta_F4,SpectralEdge_Beta_F4,PeakF_Beta_F4,...,MeanP_Delta_O2,MedianF_Delta_O2,MeanF_Delta_O2,SpectralEdge_Delta_O2,PeakF_Delta_O2,MeanP_Gamma_O2,MedianF_Gamma_O2,MeanF_Gamma_O2,SpectralEdge_Gamma_O2,PeakF_Gamma_O2
0,-0.094828,0.630879,0.703047,0.777002,0.234242,-0.079722,0.070094,-0.106329,-0.080005,-0.324620,...,-0.160751,-0.187368,-0.029275,-0.166781,-0.140427,-0.096304,-0.312128,-0.070727,-0.009962,0.439267
1,-0.093065,0.969324,0.638237,0.684541,0.015677,-0.077219,0.009672,-0.115262,-0.114617,-0.208604,...,-0.165752,-0.118640,0.126236,0.046570,-0.106358,-0.096596,-0.356000,-0.026596,-0.009962,0.439267
2,-0.093065,0.224744,0.638237,0.653720,0.039962,-0.092232,0.044199,-0.052727,-0.010780,-0.292394,...,-0.164919,0.018821,0.009604,0.188807,-0.106358,-0.096813,-0.285805,-0.059695,-0.009962,0.439267
3,-0.059582,0.427814,0.216977,0.283882,-0.130034,-0.069713,-0.145702,-0.365405,-0.426128,-0.298839,...,-0.164324,-0.153003,0.320626,0.011013,-0.072288,-0.099065,-0.619229,-0.136925,-0.009962,0.439267
4,-0.094828,0.630879,0.346595,0.407162,0.622804,-0.112249,-0.111174,-0.338604,-0.356904,-0.434192,...,-0.164085,-0.153003,0.126236,-0.166781,-0.140427,-0.099596,-0.575357,-0.125893,-0.009962,0.439267
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
89091,-0.114213,0.224744,0.638237,0.622902,-0.712877,-0.132266,0.001040,-0.293936,-0.313638,-0.105478,...,-0.158013,-0.118640,-0.184785,-0.060106,-0.072288,-0.099940,-0.961428,-1.096788,-0.872147,0.439267
89092,-0.077205,0.766260,1.286333,1.362581,-0.057180,0.010355,0.320418,0.242084,0.465139,-0.331066,...,-0.106573,-0.256099,-0.262542,-0.486814,-0.174497,-0.078057,-0.101545,-0.247254,-0.022642,-0.388218
89093,-0.073680,0.698569,0.865071,0.777002,0.549948,-0.032181,0.173676,0.054477,0.119016,-0.453528,...,-0.133722,-0.187368,-0.107032,-0.273460,-0.174497,-0.089279,-0.277031,-0.236222,-0.022642,-1.104310
89094,-0.098352,0.157059,0.346595,0.253062,0.064246,-0.122258,-0.068015,-0.204599,-0.253066,-0.363293,...,-0.139318,-0.221733,-0.145907,-0.166781,-0.174497,-0.099514,-0.864910,-1.107821,-0.998939,-1.144093


#Spliting X and y to train and test sets

In [74]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.20)

In [75]:
y_train.value_counts()

1    41171
0    16924
2    13181
Name: Sleep_Stage, dtype: int64

#Oversampling using SMOTE

**<font color='red'>SNB:</font><font color='olive'> We used *SMOTE* in train data only, so that our test data can remain corrupted data free. The accurecy of the model will be counted only for actual data.</font>**

In [76]:
!pip install imbalanced-learn

In [77]:
from imblearn.over_sampling import SMOTE

smote = SMOTE()
X_train,y_train = smote.fit_resample(X_train,y_train)

In [78]:
y_train.value_counts()

1    41171
0    41171
2    41171
Name: Sleep_Stage, dtype: int64

In [79]:
y_test.value_counts()

1    10170
0     4366
2     3284
Name: Sleep_Stage, dtype: int64

#RandomForest

In [80]:
%%time
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier()
rf.fit(X_train, y_train)
y_pred = rf.predict(X_test)

CPU times: user 2min 17s, sys: 189 ms, total: 2min 17s
Wall time: 2min 28s


In [81]:
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test,y_pred))
print("Accurecy: ",accuracy_score(y_test, y_pred))

[[3701  657    8]
 [ 700 8805  665]
 [  13  574 2697]]
              precision    recall  f1-score   support

           0       0.84      0.85      0.84      4366
           1       0.88      0.87      0.87     10170
           2       0.80      0.82      0.81      3284

    accuracy                           0.85     17820
   macro avg       0.84      0.84      0.84     17820
weighted avg       0.85      0.85      0.85     17820

Accurecy:  0.8531425364758698


#AdaBoost

In [27]:
from sklearn.ensemble import AdaBoostClassifier
ada = AdaBoostClassifier()
ada.fit(X_train, y_train)
y_pred = ada.predict(X_test)

In [28]:
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test,y_pred))
print("Accurecy: ",accuracy_score(y_test, y_pred))

[[3619  582   52]
 [2223 5661 2349]
 [  75  701 2558]]
              precision    recall  f1-score   support

           0       0.61      0.85      0.71      4253
           1       0.82      0.55      0.66     10233
           2       0.52      0.77      0.62      3334

    accuracy                           0.66     17820
   macro avg       0.65      0.72      0.66     17820
weighted avg       0.71      0.66      0.66     17820

Accurecy:  0.6643097643097643


#Gradient_Boosting

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier
grd = GradientBoostingClassifier()
grd.fit(X_train, y_train)
y_pred = grd.predict(X_test)

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test,y_pred))
print("Accurecy: ",accuracy_score(y_test, y_pred))